In [ ]:
import pandas as pd

In [ ]:
with open('input.txt') as f:
	real = f.read()

In [ ]:
test = """seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4"""

In [ ]:
def parse_maps(raw_input):
	sections = raw_input.split('\n\n')
	raw_seeds, raw_maps = sections[0], sections[1:]

	_, raw_seed_nums = raw_seeds.split(':')
	seed_nums = [int(s.strip()) for s in raw_seed_nums.split(' ') if s.isnumeric()]

	maps = []
	for raw_map in raw_maps:
		map_name, raw_mappings = raw_map.split(' map:\n')
		map_source_type, map_dest_type = map_name.split('-to-')

		mappings = [tuple(int(s) for s in mapping.split(' ')) for mapping in raw_mappings.strip().split('\n')]
		dict_map = {range(source_start, source_start + length): dest_start - source_start for dest_start, source_start, length in mappings}
		maps.append(((map_source_type, map_dest_type), dict_map))
	
	return seed_nums, maps


In [ ]:
def remap(value, map):
	for rng, diff in map.items():
		if value in rng:
			return value + diff
	return value


In [ ]:
def compute_mappings(raw_input):
	seed_nums, maps = parse_maps(raw_input)

	seeds = []
	for seed_num in seed_nums:
		seed = {'seed': seed_num}
		for (map_source_type, map_dest_type), map in maps:
			if map_source_type in seed:
				map_source_value = seed[map_source_type]
				seed[map_dest_type] = remap(map_source_value, map)
		seeds.append(seed)
	
	return pd.DataFrame(seeds)


In [ ]:
compute_mappings(test)

In [ ]:
print('part 1:', min(compute_mappings(real)['location']))

In [ ]:
parse_maps(test)[1]

In [ ]:
def overlap_ranges(r1, r2):
	"""returns a triple of ranges: the section of r1 to the left of r2, the section of r1 that overlaps r2, and the section of r1 to the right of r2"""
	return (
			range(r1.start, min(r2.start, r1.stop)), 
			range(max(r1.start, r2.start), min(r1.stop, r2.stop)),
			range(max(r2.stop, r1.start), r1.stop)
			)


In [ ]:
range_combos = [
		(range(1, 3), range(5, 7)),
		(range(1, 5), range(3, 7),),
		(range(3, 5), range(1, 7),),
		(range(3, 7), range(1, 5),),
		(range(5, 7), range(1, 3),),
		(range(1, 7), range(3, 5),)
	]
for a, b in range_combos:
	left, over, right = overlap_ranges(a, b)
	print(a, b, ':', left, over, right)


In [ ]:
def remap_range(in_range, map):
	ranges_to_process = [in_range]

	output = []
	while len(ranges_to_process) > 0:
		rng = ranges_to_process.pop()
		for map_rng, diff in map.items():
			left, overlap, right = overlap_ranges(rng, map_rng)
			if not overlap:
				continue
			output.append(range(overlap.start + diff, overlap.stop + diff))
			if left:
				ranges_to_process.append(left)
			if right:
				ranges_to_process.append(right)
			break
		else:
			# this range did not map to anything
			output.append(rng)
	
	return output

In [ ]:
parse_maps(test)[1]

In [ ]:
def compute_range_mappings(raw_input):
	seed_nums, maps = parse_maps(raw_input)

	seed_ranges = [range(seed_nums[i], seed_nums[i] + seed_nums[i+1]) for i in range(0, len(seed_nums), 2)]

	for (map_source_type, map_dest_type), map in maps:
		new_ranges = []
		for seed_range in seed_ranges:
			new_ranges.extend(remap_range(seed_range, map))
		seed_ranges = new_ranges
	
	return seed_ranges

In [ ]:
compute_range_mappings(test)

In [ ]:
print('part 2:', min([r.start for r in compute_range_mappings(real)]))